In [166]:
import os
import re
import pandas as pd
import datetime
import pyodbc

In [85]:
listdir_salida = os.listdir('../Salida/')

In [86]:
lanzamientos = [item for item in listdir_salida if item.find('lanzamiento') != -1]

In [87]:
lanzamientos = sorted(lanzamientos, key= lambda s: datetime.datetime.strptime(re.search('\d{4}-\d{2}-\d{2}', s).group(),
                                                                              '%Y-%m-%d').date())

In [ ]:
lanzamientos = lanzamientos[55:]

In [147]:
lanzamientos = ['../Salida/'+i for i in lanzamientos]

In [148]:
def crear_df_lanzamiento(xlsx):
    all_lanzamiento = list(map(pd.read_excel,xlsx))
    df_lanzamiento = pd.DataFrame([])
    lista_dfs = (list(map(df_lanzamiento.append,all_lanzamiento)))
    df = pd.concat(lista_dfs)
    return df

In [91]:
df = pd.read_excel('../Salida/'+lanzamientos)

In [162]:
df = crear_df_lanzamiento(lanzamientos)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
len(df),len(df.drop_duplicates()),len(df.dropna())

In [167]:
#INSERT
server = 'tcp:192.168.1.6'
database ="planeamiento"
username = 'sa'
password = 'sa'
cnxn = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) 

In [170]:
cursor = cnxn.cursor()
for index,row in df.iterrows():                                               
    try:
        cursor.execute("INSERT INTO sc_lanzamiento([sku],[pos],[talle],[color],[tipo],[marca],[sexo],[descripcion],[stock],[precio],[precio_dto],[fecha_alta],[moneda],[img],[url],[pagina_scraper]) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
                       row['sku'], 
                       row['pos'], 
                       row['talle'],
                       row['color'],
                       row['tipo'],
                       row['marca'],
                       row['sexo'],
                       row['descripcion'],
                       row['stock'],
                       row['precio'],
                       row['precio_dto'], 
                       row['fecha_alta'], 
                       row['moneda'],
                       row['img'], 
                       row['url'],
                       row['pagina_scraper']
                      ) 
        cnxn.commit()
    except Exception as e:
        
        print(e)
        print("*"*100)
        print(row)
        raise
cursor.close()
cnxn.close()